In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/internship/enhanced_health_data_5000.csv')
#print(df.head())
print(df.columns)



Index(['Patient_ID', 'Age', 'Gender', 'Height_cm', 'Weight_kg', 'BMI',
       'Disease_Type', 'Severity', 'Glucose_mg/dL', 'Blood_Pressure_mmHg',
       'Cholesterol_mg/dL', 'Physical_Activity_Level', 'Weekly_Exercise_Hours',
       'Daily_Caloric_Intake', 'Adherence_to_Diet_Plan',
       'Dietary_Nutrient_Imbalance_Score', 'Dietary_Restrictions', 'Allergies',
       'Preferred_Cuisine', 'Diet_Recommendation', 'BMI_Category', 'Age_Group',
       'BP_Category', 'Glucose_Category', 'BMI_Age_Interaction',
       'Glucose_BMI_Ratio', 'BP_Age_Interaction', 'Exercise_BMI_Ratio',
       'BMI_Squared', 'Age_Squared', 'Glucose_Squared',
       'Cardiovascular_Risk_Score', 'Metabolic_Health_Score',
       'Lifestyle_Health_Score', 'Calories_per_Exercise_Hour',
       'Weight_Height_Ratio', 'Cholesterol_Age_Ratio', 'High_BMI_Flag',
       'High_BP_Flag', 'High_Glucose_Flag', 'High_Cholesterol_Flag',
       'Low_Exercise_Flag', 'Poor_Diet_Flag', 'Multiple_Risk_Factors'],
      dtype='object')


In [ ]:
import numpy as np
import pandas as pd
import warnings
import os
import tensorflow as tf
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.metrics import classification_report
from imblearn.over_sampling import BorderlineSMOTE
import pickle

warnings.filterwarnings('ignore')

def load_and_preprocess_data():
    """
    Loads, preprocesses, and engineers features from the dataset.
    This function now correctly handles all required features for the model.
    """
    try:
        # Load your actual data
        df = pd.read_csv('/content/drive/My Drive/internship/enhanced_health_data_5000.csv')
    except FileNotFoundError:
        print("Error: 'enhanced_health_data_5000.csv' not found.")
        print("Please ensure your CSV file is in the correct path.")
        return None, None, None

    # Calculate BMI from raw Height and Weight
    df['BMI'] = df['Weight_kg'] / (df['Height_cm'] / 100)**2

    # Feature Engineering
    df['Glucose_over_BMI'] = df['Glucose_mg/dL'] / df['BMI'].replace(0, np.nan)
    df['Chol_over_BMI'] = df['Cholesterol_mg/dL'] / df['BMI'].replace(0, np.nan)
    df['BP_over_Age'] = df['Blood_Pressure_mmHg'] / (df['Age'] + 1)
    df['BPxBMI'] = df['Blood_Pressure_mmHg'] * df['BMI']
    df['BP_flag'] = (df['Blood_Pressure_mmHg'] >= 130).astype(int)
    df['BP_stage1_flag'] = ((df['Blood_Pressure_mmHg'] >= 130) & (df['Blood_Pressure_mmHg'] < 140)).astype(int)
    df['BP_stage2_flag'] = (df['Blood_Pressure_mmHg'] >= 140).astype(int)
    df['BP_hinge_130'] = np.clip(df['Blood_Pressure_mmHg'] - 130, 0, None)
    df['BP_hinge_140'] = np.clip(df['Blood_Pressure_mmHg'] - 140, 0, None)
    df['BP_per_BMI'] = df['Blood_Pressure_mmHg'] / (df['BMI'] + 1)
    df['Log_BP'] = np.log1p(df['Blood_Pressure_mmHg'])
    df['Metabolic_Health_Score'] = 1 / (1 + np.exp(-(130 - df['Glucose_mg/dL']) / 20))
    df['Lifestyle_Health_Score'] = 0.8 - (0.1 * (df['BMI'] > 25))
    df['Cardiovascular_Risk_Score'] = (df['Age'] * 0.005) + (df['Cholesterol_mg/dL'] * 0.001) + (df['Blood_Pressure_mmHg'] * 0.002)
    df['Multiple_Risk_Factors'] = (df['BP_flag'] | (df['Glucose_mg/dL'] > 100)).astype(int)
    df['BPx_Metabolic'] = df['Blood_Pressure_mmHg'] * df['Metabolic_Health_Score']
    df['BPx_CardioRisk'] = df['Blood_Pressure_mmHg'] * df['Cardiovascular_Risk_Score']
    df['Calories_per_BMI'] = df['Daily_Caloric_Intake'] / (df['BMI'] + 1)

    # Handle categorical features using one-hot encoding
    df = pd.get_dummies(df, columns=['Gender', 'Physical_Activity_Level'], prefix=['Gender', 'Activity'])

    # --- FIX: Drop rows with missing 'Disease_Type' first
    df = df.dropna(subset=['Disease_Type']).copy()

    # Split the multi-label string into a list of labels
    df['Disease_Type'] = df['Disease_Type'].astype(str).str.split(',')
    y = df['Disease_Type']

    # --- FIX: Select features AFTER cleaning the DataFrame
    needed_features = [
        'Age', 'Height_cm', 'Weight_kg', 'BMI', 'Daily_Caloric_Intake', 'Cholesterol_mg/dL',
        'Blood_Pressure_mmHg', 'Glucose_mg/dL', 'Cardiovascular_Risk_Score',
        'Metabolic_Health_Score', 'Lifestyle_Health_Score', 'Glucose_over_BMI',
        'Chol_over_BMI', 'BP_over_Age', 'BPxBMI', 'BP_flag', 'BP_stage1_flag',
        'BP_stage2_flag', 'BP_hinge_130', 'BP_hinge_140', 'BP_per_BMI',
        'BPx_Metabolic', 'BPx_CardioRisk', 'Calories_per_BMI', 'Log_BP',
        'Gender_Female', 'Gender_Male', 'Activity_Lightly Active',
        'Activity_Moderately Active', 'Activity_Sedentary', 'Activity_Very Active'
    ]

    final_features = [col for col in needed_features if col in df.columns]
    X = df[final_features]

    # Handle infinite and missing values in features
    X = X.replace([np.inf, -np.inf], np.nan).fillna(X.median())

    return X, y, final_features

def train_and_evaluate_xgboost(X_train, y_train, X_test, y_test, mlb):
    """
    Trains and evaluates a multi-label XGBoost classifier using a one-vs-rest strategy.
    """
    if not xgb:
        print("XGBoost is not available. Skipping this model.")
        return None, None

    print("\n--- Running XGBoost Multi-label Pipeline (One-vs-Rest) ---")

    models = {}
    for i, label in enumerate(mlb.classes_):
        print(f"Training XGBoost model for: {label}...")
        y_train_label = y_train[:, i]

        smote = BorderlineSMOTE(random_state=42, k_neighbors=3, sampling_strategy='auto')
        X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train_label)

        clf = xgb.XGBClassifier(
            objective='binary:logistic',
            eval_metric='logloss',
            random_state=42
        )
        clf.fit(X_train_sm, y_train_sm)
        models[label] = clf

    y_pred_probs = []
    for label in mlb.classes_:
        y_pred_probs.append(models[label].predict_proba(X_test)[:, 1])

    y_pred_probs = np.array(y_pred_probs).T
    y_pred = (y_pred_probs > 0.5).astype(int)

    print("\nFinal Classification Report (XGBoost):")
    print(classification_report(y_test, y_pred, target_names=mlb.classes_))
    print("\n--- XGBoost Pipeline Completed ---")

    return models, 'XGBoost'

def train_and_evaluate_tensorflow(X_train, y_train, X_test, y_test, mlb):
    """
    Trains and evaluates a TensorFlow multi-label neural network.
    """
    print("\n--- Running TensorFlow Multi-label Neural Network Pipeline ---")
    smote = BorderlineSMOTE(random_state=42, k_neighbors=3)
    X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)
    num_classes = len(mlb.classes_)

    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train_sm.shape[1],)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    model.fit(
        X_train_sm, y_train_sm,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)],
        verbose=0
    )

    print("\nFinal Classification Report (TensorFlow):")
    y_pred_probs = model.predict(X_test, verbose=0)
    y_pred = (y_pred_probs > 0.5).astype(int)
    print(classification_report(y_test, y_pred, target_names=mlb.classes_))
    print("\n--- TensorFlow Pipeline Completed ---")

    return model, 'TensorFlow'

def main_pipeline():
    """
    Main function to run the full pipeline
    """
    X, y_list, feature_names = load_and_preprocess_data()
    if X is None:
        return

    mlb = MultiLabelBinarizer()
    y_binarized = mlb.fit_transform(y_list)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_binarized, test_size=0.20, random_state=42
    )

    print("--- Data Processing Complete. Ready to Train Models. ---")

    trained_model = None
    model_type = None

    while True:
        print("\n1. Press 1 to train and evaluate with Multi-label XGBoost (One-vs-Rest)")
        print("2. Press 2 to train and evaluate with Multi-label Neural Network")
        print("3. Exit")

        try:
            n = int(input("Enter your choice: "))
            if n == 1:
                trained_model, model_type = train_and_evaluate_xgboost(X_train, y_train, X_test, y_test, mlb)
            elif n == 2:
                trained_model, model_type = train_and_evaluate_tensorflow(X_train, y_train, X_test, y_test, mlb)
            elif n == 3:
                print("Exiting program.")
                break
            else:
                print("Invalid choice. Please enter 1, 2, or 3.")
        except ValueError:
            print("Invalid input. Please enter a number.")
            continue

        if trained_model:
            model_info = {
                'model': trained_model,
                'model_type': model_type,
                'scaler': scaler,
                'mlb': mlb,
                'feature_names': feature_names
            }

            output_dir = '/content/drive/My Drive/internship/models'
            os.makedirs(output_dir, exist_ok=True)
            model_path = os.path.join(output_dir, 'trained_model.pkl')

            with open(model_path, 'wb') as f:
                pickle.dump(model_info, f)
            print(f"\nModel successfully saved to '{model_path}'")
        break

if __name__ == "__main__":
    main_pipeline()

--- Data Processing Complete. Ready to Train Models. ---

1. Press 1 to train and evaluate with Multi-label XGBoost (One-vs-Rest)
2. Press 2 to train and evaluate with Multi-label Neural Network
3. Exit
Enter your choice: 1

--- Running XGBoost Multi-label Pipeline (One-vs-Rest) ---
Training XGBoost model for: Diabetes...
Training XGBoost model for: Hypertension...
Training XGBoost model for: Obesity...

Final Classification Report (XGBoost):
              precision    recall  f1-score   support

    Diabetes       0.77      0.80      0.78       142
Hypertension       0.85      0.90      0.87       361
     Obesity       0.81      0.85      0.83       308

   micro avg       0.82      0.86      0.84       811
   macro avg       0.81      0.85      0.83       811
weighted avg       0.82      0.86      0.84       811
 samples avg       0.83      0.86      0.84       811


--- XGBoost Pipeline Completed ---

Model successfully saved to '/content/drive/My Drive/internship/models/trained_mo

In [ ]:
import numpy as np
import pandas as pd

class DietRecommender:
    """
    Provides detailed diet recommendations based on a predicted disease type,
    including specific meal plans and calorie goals, personalized for the user.
    """
    def __init__(self):
        # Base diet plans
        self.dietplans = {
            'DIABETES': """
--- Your Personalized Diabetes Management Plan ---

**Goal:** Our primary goal is to help you stabilize your blood sugar levels and feel more energized throughout the day. This plan focuses on balancing your meals to prevent sharp glucose spikes and crashes.

* **Carbohydrates:** Think "complex, not simple." Focus on high-fiber carbs like whole grains (quinoa, brown rice, oats), legumes, and vegetables. These are digested slowly, providing steady energy. Let's minimize refined grains (white bread, pasta) and all sugary drinks.
* **Protein:** Let's make protein a star in every meal. Lean protein from sources like chicken, fish, tofu, and beans will help you feel full and manage your blood sugar effectively.
* **Healthy Fats:** Good fats are your friends! Incorporate healthy fats from avocados, nuts, seeds, and olive oil to support heart health and satiety.
* **Meal Timing:** Let's aim for consistent meal times. Regular meals help regulate insulin and prevent extreme blood sugar fluctuations.

**Your Plan at a Glance:**
This plan is designed to be easily adaptable. Remember to listen to your body and adjust portion sizes as needed.
""",
            'HYPERTENSION': """
--- Your Personalized Hypertension Management Plan ---

**Goal:** Our main focus here is to lower your blood pressure. The key to this is managing your sodium intake and boosting foods that naturally support healthy blood flow.

* **Sodium Control:** Let's become "salt detectives." The goal is to reduce processed foods, fast food, and canned goods, which are often hidden sources of sodium. Instead of salt, let's explore a world of flavor with herbs, spices, lemon juice, and vinegar.
* **Potassium Power:** Potassium is a crucial mineral for balancing sodium levels. We'll increase your intake of potassium-rich foods like bananas, sweet potatoes, spinach, and tomatoes.
* **Fiber:** Fiber is your ally! Foods rich in fiber, such as whole grains and vegetables, help support a healthy circulatory system.
* **Hydration:** Staying well-hydrated is essential. Let's focus on drinking plenty of water and unsweetened beverages.

**Your Plan at a Glance:**
This plan is designed to be flavorful and satisfying, proving that healthy eating doesn't have to be boring.
""",
            'OBESITY': """
--- Your Personalized Weight Management Plan ---

**Goal:** Our plan is to create a healthy caloric deficit that leads to sustainable weight loss. This isn't about restriction; it's about nourishing your body with nutrient-dense foods to feel full and energized.
Your estimated daily calorie goal for a healthy weight loss is **{calories}** calories.

* **Calorie-Density:** Let's choose foods that are big in volume but low in calories. Think leafy greens, high-fiber vegetables, and lean proteins. These foods fill you up without adding unnecessary calories.
* **Protein:** We'll ensure every meal contains a source of lean protein. Protein helps curb cravings and preserves muscle mass as you lose weight.
* **Fiber:** Fiber is key to feeling full for longer. We'll add plenty of high-fiber foods to your meals to aid digestion and manage hunger.
* **Sugars:** Let's replace sugary drinks, sweets, and refined snacks with healthier options like whole fruits, which provide natural sweetness and fiber.

**Your Plan at a Glance:**
This plan is your roadmap to feeling lighter, healthier, and more in control of your energy.
""",
            'DIABETES_OBESITY': """
--- Personalized Diet Plan for Diabetes & Obesity ---

**Goal:** Our dual focus is on both controlling blood sugar and managing your weight through smart, balanced eating. This plan combines the principles of low-glycemic eating with a healthy caloric deficit.
Your estimated daily calorie goal for a healthy weight loss is **{calories}** calories.

* **Low Glycemic Index Carbs:** Prioritize complex carbs that don't spike blood sugar, such as whole grains, legumes, and most vegetables.
* **Lean Protein & Healthy Fats:** These promote fullness and support weight management without negatively impacting blood sugar.
* **No Sugary Drinks:** Eliminate all sugary beverages and refined snacks, as they contribute to both weight gain and blood sugar fluctuations.
""",
            'HYPERTENSION_DIABETES': """
--- Personalized Diet Plan for Hypertension & Diabetes ---

**Goal:** This plan addresses two critical health areas simultaneously: blood pressure and blood sugar. Our strategy focuses on a diet that is low in sodium and rich in nutrients that help regulate both conditions.

* **Sodium & Sugar Control:** Drastically reduce salt intake and avoid simple sugars. Focus on natural, whole foods.
* **Potassium-rich Foods:** Increase intake of potassium from fruits and vegetables to help manage blood pressure.
* **Complex Carbohydrates:** Choose high-fiber, complex carbs to help with blood sugar regulation and maintain a feeling of fullness.
* **Lean Protein:** Incorporate lean protein at every meal to help stabilize blood sugar levels.
""",
            'DIABETES_UNDERWEIGHT': """
--- Personalized Diet Plan for Diabetes (Underweight) ---

**Goal:** Our plan is to help you gain weight healthily while stabilizing your blood sugar levels. We will focus on nutrient-dense, calorie-rich foods that provide consistent energy without causing a sugar spike.

* **Calorie-Density:** Eat larger portions and incorporate healthy fats from nuts, seeds, and avocados to increase caloric intake.
* **Meal Timing:** Eat frequent, smaller meals throughout the day to support a steady energy supply.
* **Complex Carbs:** Use nutrient-dense carbohydrates like sweet potatoes, oats, and whole-wheat bread to fuel your body without causing sharp blood sugar spikes.
* **Protein:** Ensure sufficient protein intake from lean meats, dairy, or plant-based sources to support muscle and tissue growth.
""",
            'NORMAL': """
--- General Healthy Diet Plan ---

**Goal:** This plan is your guide to maintaining good health and preventing future disease. It's about balance, variety, and mindful eating.

* **Balanced Meals:** Eat a variety of fruits, vegetables, whole grains, lean proteins, and healthy fats.
* **Portion Control:** Eat mindfully and listen to your body's hunger and fullness cues.
* **Hydration:** Drink plenty of water.
"""
        }

        self.meal_plans = {
            'DIABETES': {
                'VEGETARIAN': {
                    'Breakfast': 'A bowl of whole-grain oatmeal, topped with a handful of berries, unsalted nuts, and a sprinkle of chia seeds. This combination provides a great balance of fiber and healthy fats.',
                    'Lunch': 'Hearty lentil soup with a side of mixed greens. The lentils offer slow-digesting carbs and protein, while the greens provide vitamins and fiber.',
                    'Dinner': 'A colorful stir-fry with extra-firm tofu, broccoli, bell peppers, and snap peas, served with a small portion of brown rice. Season with low-sodium soy sauce or ginger.'
                },
                'NORMAL': {
                    'Breakfast': 'A two-egg scramble with fresh spinach and mushrooms, served with one slice of whole-wheat toast. A perfect mix of protein and fiber to start your day.',
                    'Lunch': 'A generous salad with a grilled chicken breast, mixed greens, cucumber, tomatoes, and a light vinaigrette dressing. It\'s a lean, clean, and filling meal.',
                    'Dinner': 'Baked salmon fillet seasoned with lemon and dill, served with a large side of steamed broccoli and a half cup of quinoa. This meal is rich in Omega-3 fatty acids.'
                },
                'UNDERWEIGHT': {
                    'Breakfast': 'Full-fat yogurt with mixed nuts, seeds, and granola.',
                    'Lunch': 'Quinoa bowl with chickpeas, avocado, and olive oil dressing.',
                    'Dinner': 'A large portion of baked chicken breast with sweet potatoes and green beans.'
                }
            },
            'HYPERTENSION': {
                'NORMAL': {
                    'Breakfast': 'Unsalted oatmeal with sliced banana.',
                    'Lunch': 'Large green salad with grilled fish and no-salt added dressing.',
                    'Dinner': 'Baked chicken with steamed vegetables seasoned with herbs.'
                },
                'VEGETARIAN': {
                    'Breakfast': 'Plain yogurt with unsalted nuts and fruit.',
                    'Lunch': 'Lentil and vegetable curry (no added salt) with brown rice.',
                    'Dinner': 'Black bean burgers on a whole-wheat bun with a side of sweet potato fries.'
                }
            },
            'OBESITY': {
                'NORMAL': {
                    'Breakfast': 'Greek yogurt with a handful of berries.',
                    'Lunch': 'A large bowl of mixed greens with grilled lean protein.',
                    'Dinner': 'Baked cod with a side of steamed asparagus and a sprinkle of lemon juice.'
                },
                'VEGETARIAN': {
                    'Breakfast': 'A smoothie with spinach, a scoop of plant-based protein powder, and half a banana.',
                    'Lunch': 'Large chickpea and quinoa salad with low-fat dressing.',
                    'Dinner': 'Tofu and vegetable stir-fry with a small amount of sesame oil.'
                }
            },
            'DIABETES_OBESITY': {
                'NORMAL': {
                    'Breakfast': 'A vegetable omelet with low-fat cheese.',
                    'Lunch': 'Chicken and vegetable skewers with a small portion of brown rice.',
                    'Dinner': 'Grilled fish with a large serving of steamed broccoli and cauliflower.'
                },
                'VEGETARIAN': {
                    'Breakfast': 'Scrambled tofu with spinach and a slice of whole-wheat toast.',
                    'Lunch': 'Black bean soup with a side of steamed greens.',
                    'Dinner': 'Roasted vegetables (broccoli, bell peppers) with a small portion of lentils.'
                }
            },
            'HYPERTENSION_DIABETES': {
                'NORMAL': {
                    'Breakfast': 'Oatmeal with berries and a sprinkle of cinnamon.',
                    'Lunch': 'Brown rice and black bean bowl with unsalted vegetables.',
                    'Dinner': 'Baked salmon with steamed greens and a small sweet potato.'
                },
                'VEGETARIAN': {
                    'Breakfast': 'Quinoa porridge with almond milk and berries.',
                    'Lunch': 'Unsalted veggie and tofu stir-fry with a side of brown rice.',
                    'Dinner': 'Lentil loaf with roasted carrots and a large side salad.'
                }
            },
            'NORMAL': {
                'NORMAL': {
                    'Breakfast': 'Avocado toast on whole-wheat bread with a boiled egg.',
                    'Lunch': 'Turkey and veggie wrap on a whole-wheat tortilla.',
                    'Dinner': 'Lean steak with roasted potatoes and a side salad.'
                },
                'VEGETARIAN': {
                    'Breakfast': 'Whole-wheat toast with mashed avocado and tomato slices.',
                    'Lunch': 'Hummus and veggie sandwich on whole-grain bread.',
                    'Dinner': 'Baked falafel with a large Greek salad.'
                }
            }
        }

    def calculate_user_tdee(self, user_data):
        """
        Calculates the user's Total Daily Energy Expenditure (TDEE).
        """
        activity_multipliers = {
            'sedentary': 1.2,
            'lightly active': 1.375,
            'moderately active': 1.55,
            'very active': 1.725
        }
        activity_level = user_data.get('Activity_Level', 'sedentary').lower()
        if activity_level not in activity_multipliers:
            activity_level = 'sedentary'
        age = user_data.get('Age', 30)
        height_cm = user_data.get('Height_cm', 170)
        weight_kg = user_data.get('Weight_kg', 70)
        gender = user_data.get('Gender', 'male').lower()
        if gender == 'female':
            bmr = (10 * weight_kg) + (6.25 * height_cm) - (5 * age) - 161
        else:
            bmr = (10 * weight_kg) + (6.25 * height_cm) - (5 * age) + 5
        tdee = bmr * activity_multipliers[activity_level]
        return tdee

    def get_diet_recommendation(self, diseases, user_data):
        """
        Takes a list of diseases and returns a combined, detailed diet plan
        based on conditions and dietary preferences, allergies, and restrictions.
        """
        main_condition = 'NORMAL'
        has_diabetes = 'Diabetes' in diseases
        has_obesity = 'Obesity' in diseases
        has_hypertension = 'Hypertension' in diseases

        if has_diabetes and has_obesity:
            main_condition = 'DIABETES_OBESITY'
        elif has_diabetes and has_hypertension:
            main_condition = 'HYPERTENSION_DIABETES'
        elif has_diabetes and user_data.get('BMI', 22) < 18.5:
            main_condition = 'DIABETES_UNDERWEIGHT'
        elif has_diabetes:
            main_condition = 'DIABETES'
        elif has_obesity:
            main_condition = 'OBESITY'
        elif has_hypertension:
            main_condition = 'HYPERTENSION'

        plan = self.dietplans[main_condition]

        if main_condition in ['OBESITY', 'DIABETES_OBESITY']:
            tdee = self.calculate_user_tdee(user_data)
            calorie_goal = tdee - 500
            plan = plan.format(calories=int(calorie_goal))

        preference = user_data.get('Dietary_Preference', 'NORMAL').upper()

        meal_plan_key = 'NORMAL'
        if main_condition in self.meal_plans and preference in self.meal_plans[main_condition]:
            meal_plan_key = preference

        meal_plan = self.meal_plans.get(main_condition, {}).get(meal_plan_key, self.meal_plans['NORMAL']['NORMAL'])

        final_output = plan

        final_output += "\n\n--- Sample Meal Plan ---\n"
        final_output += f"**Breakfast:** {meal_plan['Breakfast']}\n"
        final_output += f"**Lunch:** {meal_plan['Lunch']}\n"
        final_output += f"**Dinner:** {meal_plan['Dinner']}"

        final_output += "\n\n--- Personal Dietary Notes ---\n"

        cuisine = user_data.get('Preferred_Cuisine', '').strip()
        if cuisine:
            final_output += f"**Preferred Cuisine:** The provided plan can be adapted to a **{cuisine}**-style diet by incorporating relevant spices, cooking methods, and ingredients. For example, use olive oil, fresh vegetables, and lean protein for Mediterranean cuisine.\n"
        else:
            final_output += "**Preferred Cuisine:** No specific preference noted. The plan is a general guideline.\n"

        restrictions = user_data.get('Dietary_Restrictions', '').strip()
        if restrictions:
            final_output += f"**Dietary Restrictions:** The plan has been designed to honor your restriction(s) of **{restrictions}**. For example, if low-carb, substitute grains with vegetables.\n"
        else:
            final_output += "**Dietary Restrictions:** None noted. The plan provides general advice.\n"

        allergies = user_data.get('Allergies', '').strip()
        if allergies:
            final_output += f"**Allergies:** This is critically important. Please ensure all meals and ingredients are free of **{allergies}** to prevent adverse reactions.\n"
            final_output += "\n**⚠️ CRITICAL SAFETY WARNING:** Always read food labels and consult a healthcare professional. This plan is not a substitute for professional medical advice.\n"
        else:
            final_output += "**Allergies:** None noted. Always check ingredients to be safe.\n"

        return final_output

In [ ]:
import numpy as np
import pandas as pd
import pickle
import os

def get_user_data():
    print("--- Enter Your Health Parameters ---")
    data = {}
    try:
        data['Age'] = float(input("Enter your Age (e.g., 45): "))
        data['Gender'] = input("Enter your gender (Male/Female): ")
        data['Height_cm'] = float(input("Enter your height in cm (e.g., 175): "))
        data['Weight_kg'] = float(input("Enter your weight in kg (e.g., 70): "))
        data['BMI'] = float(input("Enter your BMI (e.g., 32): "))
        data['Daily_Caloric_Intake'] = float(input("Enter your Daily Caloric Intake (e.g., 2500): "))
        data['Cholesterol_mg/dL'] = float(input("Enter your Cholesterol in mg/dL (e.g., 210): "))
        data['Blood_Pressure_mmHg'] = float(input("Enter your Blood Pressure in mmHg (e.g., 145): "))
        data['Glucose_mg/dL'] = float(input("Enter your Glucose in mg/dL (e.g., 115): "))
        data['Calories_per_Exercise_Hour'] = float(input("Enter your Calories per Exercise Hour (e.g., 300): "))
        data['Activity_Level'] = input("Enter your activity level(Sedentary/Lightly Active/Moderately Active/Very Active)")
    except ValueError:
        print("Invalid input. Please enter numeric values for all parameters.")
        return None

    if data['BMI'] == 0:
        print("BMI cannot be zero. Please re-enter a valid BMI.")
        return None

    # Recalculate BMI based on Height and Weight for consistency
    # This also helps if user-entered BMI is inaccurate
    data['BMI'] = data['Weight_kg'] / (data['Height_cm'] / 100)**2

    data['Cardiovascular_Risk_Score'] = (data['Age'] * 0.005) + (data['Cholesterol_mg/dL'] * 0.001) + (data['Blood_Pressure_mmHg'] * 0.002)
    data['Metabolic_Health_Score'] = 1 / (1 + np.exp(-(130 - data['Glucose_mg/dL']) / 20))
    data['Lifestyle_Health_Score'] = 0.8 - (0.1 * (data['BMI'] > 25))
    data['Multiple_Risk_Factors'] = int((data['Blood_Pressure_mmHg'] >= 130) or (data['Glucose_mg/dL'] > 100))
    data['BMI_Age_Interaction'] = data['Age'] * data['BMI']
    data['Cholesterol_Age_Ratio'] = data['Cholesterol_mg/dL'] / data['Age']
    data['BMI_Squared'] = data['BMI']**2
    data['Glucose_Squared'] = data['Glucose_mg/dL']**2
    data['Glucose_over_BMI'] = data['Glucose_mg/dL'] / data['BMI']
    data['Chol_over_BMI'] = data['Cholesterol_mg/dL'] / data['BMI']
    data['BP_over_Age'] = data['Blood_Pressure_mmHg'] / (data['Age'] + 1)
    data['BPxBMI'] = data['Blood_Pressure_mmHg'] * data['BMI']
    data['BP_flag'] = int(data['Blood_Pressure_mmHg'] >= 130)
    data['BP_stage1_flag'] = int((data['Blood_Pressure_mmHg'] >= 130) and (data['Blood_Pressure_mmHg'] < 140))
    data['BP_stage2_flag'] = int(data['Blood_Pressure_mmHg'] >= 140)
    data['BP_hinge_130'] = max(0, data['Blood_Pressure_mmHg'] - 130)
    data['BP_hinge_140'] = max(0, data['Blood_Pressure_mmHg'] - 140)
    data['BP_per_BMI'] = data['Blood_Pressure_mmHg'] / (data['BMI'] + 1)
    data['BPx_Metabolic'] = data['Blood_Pressure_mmHg'] * data['Metabolic_Health_Score']
    data['BPx_CardioRisk'] = data['Blood_Pressure_mmHg'] * data['Cardiovascular_Risk_Score']
    data['Calories_per_BMI'] = data['Daily_Caloric_Intake'] / (data['BMI'] + 1)
    data['Log_BP'] = np.log1p(data['Blood_Pressure_mmHg'])
    data['Dietary_Preference'] = input("Enter your dietary preference (Normal/Vegetarian): ")
    data['Preferred_Cuisine'] = input("Enter your preferred cuisine (e.g., Mediterranean, Asian, American): ")
    data['Dietary_Restrictions'] = input("Enter any dietary restrictions (e.g., Gluten-Free, Low-Carb): ")
    data['Allergies'] = input("Enter any allergies (e.g., Peanuts, Dairy): ")
    return data

def make_prediction_and_recommendation():
    print("\n--- Running Prediction and Diet Recommendation Pipeline ---")
    model_path = '/content/drive/My Drive/internship/models/trained_model.pkl'

    if not os.path.exists(model_path):
        print(f"Error: Model file '{model_path}' not found.")
        print("Please run `training_pipeline_updated.py` first to train and save the model.")
        return

    print(f"Loading model from '{model_path}'...")
    with open(model_path, 'rb') as f:
        model_info = pickle.load(f)

    trained_model = model_info['model']
    model_type = model_info['model_type']
    scaler = model_info['scaler']
    mlb = model_info['mlb']
    feature_names = model_info['feature_names']

    print("Model loaded successfully!")

    user_data = get_user_data()
    if user_data is None:
        return

    user_df = pd.DataFrame([user_data])

    # Convert 'Gender' and 'Activity_Level' to one-hot encoded columns
    user_df = pd.get_dummies(user_df, columns=['Gender', 'Activity_Level'], prefix=['Gender', 'Activity'])

    # Align features with the model's training features
    user_data_aligned = user_df.reindex(columns=feature_names, fill_value=0)
    user_data_scaled = scaler.transform(user_data_aligned)

    if model_type == 'XGBoost':
        y_pred = []
        for label, model in trained_model.items():
            y_pred.append(model.predict_proba(user_data_scaled)[:, 1])
        y_pred_probs = np.array(y_pred).T[0]
    else:
        y_pred_probs = trained_model.predict(user_data_scaled, verbose=0)[0]

    predicted_diseases_binary = (y_pred_probs > 0.5).astype(int)
    predicted_diseases_list = mlb.inverse_transform(predicted_diseases_binary.reshape(1, -1))[0]
    predicted_diseases = list(predicted_diseases_list)

    print("\n" + "="*50)
    print("      *** Personalized Health Report & Recommendation ***")
    print("="*50)

    print("\n--- Your Health Snapshot ---")
    print(f"**Age:** {user_data['Age']} years")
    print(f"**Gender:** {user_data['Gender']}")
    print(f"**Height:** {user_data['Height_cm']} cm")
    print(f"**Weight:** {user_data['Weight_kg']} kg")
    print(f"**BMI:** {user_data['BMI']:.2f}")
    print(f"**Blood Pressure:** {user_data['Blood_Pressure_mmHg']} mmHg")
    print(f"**Glucose Level:** {user_data['Glucose_mg/dL']} mg/dL")
    print(f"**Cholesterol:** {user_data['Cholesterol_mg/dL']} mg/dL")
    print(f"**Activity Level:** {user_data['Activity_Level']}")

    print("\n--- Prediction Analysis ---")
    print("Based on our analysis, here is your risk profile:")
    for i, disease in enumerate(mlb.classes_):
        confidence_score = y_pred_probs[i]
        print(f"- **{disease.capitalize()}**: {confidence_score:.2f} confidence")

    if not predicted_diseases:
        print("\nNote: No specific conditions met the confidence threshold (0.5), but the risk scores above provide a detailed view of your health profile.")

    print("\n--- Personalized Diet Plan ---")
    recommender = DietRecommender()
    diet_plan = recommender.get_diet_recommendation(predicted_diseases, user_data)
    print(diet_plan)

    print("\n" + "-"*50)
    print("Disclaimer: This information is for educational purposes only and is not a substitute for professional medical advice. Always consult a qualified healthcare professional before making any decisions about your health.")
    print("-"*50 + "\n")

if __name__ == "__main__":
    make_prediction_and_recommendation()


--- Running Prediction and Diet Recommendation Pipeline ---
Loading model from '/content/drive/My Drive/internship/models/trained_model.pkl'...
Model loaded successfully!
--- Enter Your Health Parameters ---
Enter your Age (e.g., 45): 48
Enter your gender (Male/Female): Male
Enter your height in cm (e.g., 175): 175
Enter your weight in kg (e.g., 70): 90
Enter your BMI (e.g., 32): 29.39
Enter your Daily Caloric Intake (e.g., 2500): 2800
Enter your Cholesterol in mg/dL (e.g., 210): 225
Enter your Blood Pressure in mmHg (e.g., 145): 135
Enter your Glucose in mg/dL (e.g., 115): 185
Enter your Calories per Exercise Hour (e.g., 300): 250
Enter your activity level(Sedentary/Lightly Active/Moderately Active/Very Active)Sedentary
Enter your dietary preference (Normal/Vegetarian): Normal
Enter your preferred cuisine (e.g., Mediterranean, Asian, American): American
Enter any dietary restrictions (e.g., Gluten-Free, Low-Carb): None
Enter any allergies (e.g., Peanuts, Dairy): None

      *** Perso